In [1]:
import napari
import pyclesperanto_prototype as cle
import pandas as pd
from skimage.io import imread
from aicsimageio import AICSImage

files_location = 'C:/Users/ryans/OneDrive/Documents/Bia POL stuff/Napari Plugin Development/ncp example timelapse data/'
image_name = 't000-t017-for-edited-cropped.tif'
label_name = 't000-t017-labels-edited-cropped.tif'
image = imread(files_location+ image_name)
label = imread(files_location+ label_name)

In [2]:
lazy_stack = AICSImage(image,chunk_dims=["T", "Z", "Y", "X"])
lazy_stack.shape

(1, 17, 59, 182, 366)

In [3]:
dask_array = lazy_stack.get_image_dask_data()
dask_array

dask.array<transpose, shape=(1, 17, 59, 182, 366), dtype=uint16, chunksize=(1, 17, 59, 182, 366), chunktype=numpy.ndarray>

In [4]:
viewer = napari.Viewer()
viewer.add_image(dask_array)

C:\Users\ryans\Anaconda3\envs\ncp-dev\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'dask_array' at 0x2e1fdd6e2e0>

This works like a treat for images already as a stack, super easy to set up and works with napari. How to deal with folders of files is suggested on the napari page so we'll try that with the tribolium data from the master thesis:

In [5]:
from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
import dask.array as da
from glob import glob

location = "C:/Users/ryans/Documents/output data (big)/Lund Gastrulation Correct voxel sizes/"
filenames = sorted(glob(location + "rescaled t =*.tif"), key=alphanumeric_key)
# read the first file to get the shape and dtype
# ASSUMES THAT ALL FILES SHARE THE SAME SHAPE/TYPE
sample = imread(filenames[0])
sample.shape

(302, 1024, 512)

In [6]:
lazy_imread = delayed(imread)  # lazy reader
lazy_arrays = [lazy_imread(fn) for fn in filenames]
dask_arrays = [
    da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
    for delayed_reader in lazy_arrays
]
# Stack into one large dask.array
stack = da.stack(dask_arrays, axis=0)
stack.shape  # (nfiles, nz, ny, nx)

# in jupyter notebook the repr of a dask stack provides a useful visual:
stack

dask.array<stack, shape=(25, 302, 1024, 512), dtype=float32, chunksize=(1, 302, 1024, 512), chunktype=numpy.ndarray>

In [14]:
viewer2 = napari.Viewer()
viewer2.add_image(stack, contrast_limits=[0,2000], multiscale=False)

C:\Users\ryans\Anaconda3\envs\ncp-dev\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'stack' at 0x2e19b7e5550>

So this works but is far from actually feeling usable. I rememberred where a stack was of this dataset so let's see if that changes anything (it should be said that it has ca 50 less z slices makig it slightly more manageable:

In [8]:
data_location2 = "C:/Users/ryans/Documents/output data (big)/Lund Gastrulation Labels/"
file_location = data_location2 + 'rescaled stack.tif'
from aicsimageio import readers
lund_stack = AICSImage(file_location,chunk_dims=["T", "Z", "Y", "X"],reader=readers.TiffReader)

In [9]:
lund_stack.shape

C:\Users\ryans\Anaconda3\envs\ncp-dev\lib\site-packages\aicsimageio\aics_image.py:533: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  return self.xarray_dask_data.shape


(1, 25, 252, 1024, 512)

In [10]:
lund_aics_dask_array = lund_stack.get_image_dask_data()
lund_aics_dask_array

dask.array<transpose, shape=(1, 25, 252, 1024, 512), dtype=float32, chunksize=(1, 1, 252, 1024, 512), chunktype=numpy.ndarray>

In [13]:
viewer3 = napari.Viewer()
viewer3.add_image(lund_aics_dask_array, contrast_limits=[0,2000], multiscale=False)

C:\Users\ryans\Anaconda3\envs\ncp-dev\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'lund_aics_dask_array' at 0x2e193f13ac0>

Viewing doesn't differ too much from the folder technique so maybe we'll just stick to the aics way when you have a stack and the dask way to when we have a folder of images